# Informational Architecture

In [ ]:
using Distributed
addprocs(16);

In [1]:
using Distributed, ClusterManagers
addprocs(SlurmManager(160))

connecting to worker 1 out of 160
connecting to worker 2 out of 160
connecting to worker 3 out of 160
connecting to worker 4 out of 160
connecting to worker 5 out of 160
connecting to worker 6 out of 160
connecting to worker 7 out of 160
connecting to worker 8 out of 160
connecting to worker 9 out of 160
connecting to worker 10 out of 160
connecting to worker 11 out of 160
connecting to worker 12 out of 160
connecting to worker 13 out of 160
connecting to worker 14 out of 160
connecting to worker 15 out of 160
connecting to worker 16 out of 160
connecting to worker 17 out of 160
connecting to worker 18 out of 160
connecting to worker 19 out of 160
connecting to worker 20 out of 160
connecting to worker 21 out of 160
connecting to worker 22 out of 160
connecting to worker 23 out of 160
connecting to worker 24 out of 160
connecting to worker 25 out of 160
connecting to worker 26 out of 160
connecting to worker 27 out of 160
connecting to worker 28 out of 160
connecting to worker 29 out o

160-element Array{Int64,1}:
   2
   3
   4
   5
   6
   7
   8
   9
  10
  11
  12
  13
  14
   ⋮
 150
 151
 152
 153
 154
 155
 156
 157
 158
 159
 160
 161

In [2]:
@everywhere begin
    using Pkg
    Pkg.activate("..")
end

 Activating environment at `~/biobot-punctures/Project.toml`


In [50]:
using CSV, DataFrames, Distributions, DrWatson, LinearAlgebra, Plots, StatsPlots
include(srcdir("information.jl"));

In [51]:
analysisdir = datadir("2020-12-11", "analysis")
readdir(analysisdir)

6-element Array{String,1}:
 "2020-11-11_1"
 "2020-11-11_2"
 "2020-11-11_3"
 "2020-11-17_1"
 "2020-11-17_2"
 "2020-11-18"

In [52]:
datasetdir = joinpath(analysisdir, "2020-11-11_1");
phase1dir = joinpath(datasetdir, "phase_1");
phase2dir = joinpath(datasetdir, "phase_2");

In [53]:
phase1_series = CSV.File(joinpath(phase1dir, "series_trimmed.csv")) |> DataFrame;
phase1_com = CSV.File(joinpath(phase1dir, "com_trimmed.csv")) |> DataFrame;
phase1_com[!,:phase] .= 1;
phase1_image = load(joinpath(phase2dir, "video.tif"));
phase1_δ = transpose(Array(phase1_com[:,[:x,:y]])) |> distance;

In [54]:
phase2_series = CSV.File(joinpath(phase2dir, "series_trimmed.csv")) |> DataFrame;
phase2_com = CSV.File(joinpath(phase2dir, "com_trimmed.csv")) |> DataFrame;
phase2_com[!,:phase] .= 2;
phase2_image = load(joinpath(phase2dir, "video.tif"));
phase2_δ = transpose(Array(phase2_com[:,[:x,:y]])) |> distance;

In [55]:
phase1_c = hcat([g.c for g in groupby(phase1_series, :cellid)]...);
phase2_c = hcat([g.c for g in groupby(phase2_series, :cellid)]...);

In [56]:
𝒩 = Normal(0, 1e-8)
phase1_c .+= rand(𝒩, size(phase1_c)...);
phase2_c .+= rand(𝒩, size(phase2_c)...);

# Mutual Information

In [57]:
nn1, nn2, nperm = findnn(phase1_c), findnn(phase2_c), 100

(9, 5, 100)

In [58]:
@time phase1_mi_matrix = mi_matrix(phase1_c; nperm, nn=nn1);
@time phase2_mi_matrix = mi_matrix(phase2_c; nperm, nn=nn2);

 21.472577 seconds (7.25 M allocations: 310.567 MiB, 0.99% gc time)
  9.068392 seconds (6.37 M allocations: 267.867 MiB, 1.10% gc time)


In [59]:
mi = DataFrame(phase=Int[],
               source=Int[], target=Int[],
               source_x=Float64[], source_y=Float64[], target_x=Float64[], target_y=Float64[],
               distance=Float64[],
               value=Float64[], p=Float64[], nperm=Int[])
for i in 1:size(phase1_mi_matrix, 1), j in i:size(phase1_mi_matrix, 2)
    source, source_x, source_y = phase1_com[i, [:cellid, :x, :y]]
    target, target_x, target_y = phase1_com[j, [:cellid, :x, :y]]
    δ = phase1_δ[i, j]
    sig = phase1_mi_matrix[i, j]
    push!(mi, [1, source, target, source_x, source_y, target_x, target_y, δ, sig.value, sig.p, sig.nperm])
end
for i in 1:size(phase2_mi_matrix, 1), j in i:size(phase2_mi_matrix, 2)
    source, source_x, source_y = phase2_com[i, [:cellid, :x, :y]]
    target, target_x, target_y = phase2_com[j, [:cellid, :x, :y]]
    δ = phase2_δ[i, j]
    sig = phase2_mi_matrix[i, j]
    push!(mi, [2, source, target, source_x, source_y, target_x, target_y, δ, sig.value, sig.p, sig.nperm])
end

In [60]:
let params = @dict nperm nn1 nn2
    fname = joinpath(datasetdir, savename("mi", params, "csv"))
    CSV.write(fname, mi)
end;

# Active Information

In [61]:
phase1_ks, phase1_τs = estimate_embedding(phase1_c);

In [62]:
phase2_ks, phase2_τs = estimate_embedding(phase2_c);

In [63]:
ai = DataFrame(phase=Int[],
               source=Int[],
               source_x=Float64[], source_y=Float64[],
               k=Int[], τ=Int[],
               value=Float64[])
for i in 1:size(phase1_c, 2)
    source, source_x, source_y = phase1_com[i, [:cellid, :x, :y]]
    k = phase1_ks[i]
    τ = phase1_τs[i]
    value = activeinfo(phase1_c[:, i]; k, τ)
    push!(ai, [1, source, source_x, source_y, k, τ, value])
end

for i in 1:size(phase2_c, 2)
    source, source_x, source_y = phase2_com[i, [:cellid, :x, :y]]
    k = phase2_ks[i]
    τ = phase2_τs[i]
    value = activeinfo(phase2_c[:, i]; k, τ)
    push!(ai, [2, source, source_x, source_y, k, τ, value])
end

In [64]:
ai

,phase,source,source_x,source_y,k,τ,value
,Int64,Int64,Float64,Float64,Int64,Int64,Float64
1,1,18,72.8974,145.022,1,1,2.04576
2,1,19,73.0881,151.173,1,1,3.29359
3,1,20,64.5041,159.257,1,1,3.48346
4,1,21,62.3608,182.609,1,1,3.71343
5,1,22,62.6136,197.611,1,1,4.35894
6,1,31,92.4286,145.281,1,1,3.2039
7,1,32,81.4372,158.033,1,1,3.75479
8,1,33,90.8739,154.183,1,1,3.81528
9,1,34,86.8677,175.241,1,1,2.34705


In [65]:
CSV.write(joinpath(datasetdir, "ai.csv"), ai);

# Transfer Entropy

In [66]:
nperm = 10

10

In [67]:
@time phase1_te_matrix, phase1_delay_matrix = te_matrix(phase1_c, phase1_ks, phase1_τs, 1:10; nperm, nn=nn1);

 78.634892 seconds (14.64 M allocations: 707.681 MiB, 0.39% gc time)


In [68]:
@time phase2_te_matrix, phase2_delay_matrix = te_matrix(phase2_c, phase2_ks, phase2_τs, 1:10; nperm, nn=nn2);

 33.570808 seconds (13.12 M allocations: 616.060 MiB, 0.75% gc time)


In [72]:
te = DataFrame(phase=Int[],
               source=Int[], target=Int[],
               source_x=Float64[], source_y=Float64[], target_x=Float64[], target_y=Float64[],
               distance=Float64[],
               k=Int[], τ=Int[], delay=Int[],
               value=Float64[], p=Float64[], nperm=Int[])
for i in 1:size(phase1_te_matrix, 1), j in 1:size(phase1_te_matrix, 2)
    source, source_x, source_y = phase1_com[i, [:cellid, :x, :y]]
    target, target_x, target_y = phase1_com[j, [:cellid, :x, :y]]
    δ = phase1_δ[i, j]
    sig = phase1_te_matrix[i, j]
    k = phase1_ks[j]
    τ = phase1_τs[j]
    delay = phase1_delay_matrix[i, j]
    push!(te, [1, source, target, source_x, source_y, target_x, target_y, δ, k, τ, delay, sig.value, sig.p, sig.nperm])
end
for i in 1:size(phase2_te_matrix, 1), j in 1:size(phase2_te_matrix, 2)
    source, source_x, source_y = phase2_com[i, [:cellid, :x, :y]]
    target, target_x, target_y = phase2_com[j, [:cellid, :x, :y]]
    δ = phase2_δ[i, j]
    sig = phase2_te_matrix[i, j]
    k = phase2_ks[j]
    τ = phase2_τs[j]
    delay = phase2_delay_matrix[i, j]
    push!(te, [2, source, target, source_x, source_y, target_x, target_y, δ, k, τ, delay, sig.value, sig.p, sig.nperm])
end

In [73]:
te

,phase,source,target,source_x,source_y,target_x,target_y,distance,k
,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Int64
1,1,18,18,72.8974,145.022,72.8974,145.022,0.0,1
2,1,18,19,72.8974,145.022,73.0881,151.173,6.15381,1
3,1,18,20,72.8974,145.022,64.5041,159.257,16.5256,1
4,1,18,21,72.8974,145.022,62.3608,182.609,39.036,1
5,1,18,22,72.8974,145.022,62.6136,197.611,53.5849,1
6,1,18,31,72.8974,145.022,92.4286,145.281,19.533,1
7,1,18,32,72.8974,145.022,81.4372,158.033,15.5631,1
8,1,18,33,72.8974,145.022,90.8739,154.183,20.1764,1
9,1,18,34,72.8974,145.022,86.8677,175.241,33.2919,1


In [74]:
let params = @dict nperm nn1 nn2
    fname = joinpath(datasetdir, savename("te", params, "csv"))
    CSV.write(fname, te)
end;